In [1]:
data_var = '2023-12-29'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9796,2023-12-29,Brasil Nbb,17:30,Paulistano,São José,1.47,2.56,150.5,1.86,1.84,-5.5,2.01,1.72,8CMqCPgP,0.680272,0.390625,0.537634,0.543478,0.070897,1.0,0.2,NaN,NaN,103.240,7.839120,0.075931,129.142,27.623664,0.213901,130.496,132.034,107.64,153.18,0.0,0.0,0.0,0.0,0.382504,0.007644,0.109952,-2.67,-0.534,-0.880150,0.0,0.0,0.0,-0.61,-0.122,-12.786885,0.0,0.0,0.0
9797,2023-12-29,Brasil Nbb,19:00,Corinthians Paulista,Flamengo,3.25,1.31,159.5,1.85,1.85,6.5,2.04,1.69,U1mxgpmm,0.307692,0.763359,0.540541,0.540541,0.071051,0.8,0.6,NaN,NaN,162.658,81.753565,0.502610,110.400,12.716126,0.115182,150.926,105.778,137.28,108.24,0.0,0.0,0.0,0.0,0.601661,0.000000,0.132701,-0.75,-0.150,-15.000000,0.0,0.0,0.0,0.28,0.056,5.535714,0.0,0.0,0.0
9798,2023-12-29,Brasil Nbb,19:00,Minas,União Corinthians,1.10,6.25,158.5,1.85,1.85,-15.5,1.99,1.73,CQxsh42g,0.909091,0.160000,0.540541,0.540541,0.069091,0.4,0.4,NaN,NaN,106.062,18.862853,0.177847,229.756,84.460739,0.367611,92.500,307.370,91.56,168.84,0.0,0.0,0.0,0.0,0.990912,0.000000,0.098843,-0.55,-0.110,-0.909091,0.0,0.0,0.0,-1.14,-0.228,-23.026316,0.0,0.0,0.0
9799,2023-12-29,Brasil Nbb,19:30,Fortaleza B.C.,Brasília,1.17,4.66,157.5,1.85,1.85,-12.5,2.09,1.66,KjtkjrX5,0.854701,0.214592,0.540541,0.540541,0.069293,0.4,0.4,NaN,NaN,178.182,106.968897,0.600335,505.808,215.141857,0.425343,107.754,481.112,118.32,412.62,0.0,0.0,0.0,0.0,0.846588,0.000000,0.162163,-0.58,-0.116,-1.465517,0.0,0.0,0.0,-5.00,-1.000,-3.660000,0.0,0.0,0.0
9800,2023-12-29,Brasil Nbb,19:30,Pato,Caxias do Sul,1.46,2.59,150.5,1.85,1.85,-5.5,1.99,1.73,jHyoiOHa,0.684932,0.386100,0.540541,0.540541,0.071032,0.4,0.4,NaN,NaN,251.192,150.715095,0.600000,256.002,85.684244,0.334701,237.012,313.238,112.97,352.55,0.0,0.0,0.0,0.0,0.394583,0.000000,0.098843,5.37,1.074,0.428305,0.0,0.0,0.0,-1.14,-0.228,-6.973684,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,2023-12-29,Eua Ncaa,22:00,UMKC,Oral Roberts,2.61,1.51,143.5,1.91,1.80,3.5,1.83,1.83,jZWGqQu2,0.383142,0.662252,0.523560,0.555556,0.045393,0.0,0.6,NaN,NaN,330.024,435.177819,1.318625,400.100,394.174344,0.985190,0.000,643.374,91.02,222.00,0.0,0.0,0.0,0.0,0.377581,0.041931,0.000000,0.00,0.000,inf,0.0,0.0,0.0,-5.00,-1.000,-0.510000,0.0,0.0,0.0
9933,2023-12-29,Eua Ncaa,22:00,Wisc. Milwaukee,Robert Morris,1.59,2.35,153.5,1.83,1.83,-3.5,1.80,1.91,xGSCAS71,0.628931,0.425532,0.546448,0.546448,0.054463,0.0,0.6,NaN,NaN,200.928,98.097585,0.488223,209.386,95.194972,0.454639,0.000,391.512,193.80,113.25,0.0,0.0,0.0,0.0,0.272792,0.000000,0.041931,0.00,0.000,inf,0.0,0.0,0.0,-3.49,-0.698,-1.934097,0.0,0.0,0.0
9934,2023-12-29,Eua Ncaa,22:30,SE Missouri State,Southern Indiana,1.73,2.10,141.5,1.80,1.91,-2.5,2.00,1.80,ptnkMI7E,0.578035,0.476190,0.555556,0.523560,0.054225,0.0,0.4,NaN,NaN,407.276,277.467373,0.681276,366.316,352.859026,0.963264,0.000,475.100,111.30,525.00,0.0,0.0,0.0,0.0,0.136621,0.041931,0.074432,0.00,0.000,inf,0.0,0.0,0.0,-5.00,-1.000,-1.100000,0.0,0.0,0.0
9935,2023-12-29,Eua Ncaa,22:30,Siu Edwardsville,Eastern Illinois,1.26,3.72,133.5,1.83,1.83,-8.5,1.83,1.83,bmSPFGpr,0.793651,0.268817,0.546448,0.546448,0.062468,0.0,0.4,NaN,NaN,170.302,100.890494,0.592421,145.852,60.037122,0.411630,0.000,417.206,104.52,91.68,0.0,0.0,0.0,0.0,0.698587,0.000000,0.000000,0.00,0.000,i

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Miami Heat,229.5,1.8,0.9999,Over
1,00:00,Eua Nba,Portland Trail Blazers,San Antonio Spurs,231.5,1.8,0.9999,Over
2,00:30,Eua Nba,Los Angeles Lakers,Charlotte Hornets,224.5,1.8,0.9999,Over
3,21:30,Eua Nba,Boston Celtics,Toronto Raptors,224.5,1.8,1.0000,Over
